In [ ]:
# This cell does not get executed when run with Times Square
import os
import datetime

visit_origin = os.environ.get("SCHEDVIEW_VISIT_ORIGIN", "lsstcam")
day_obs = int(
    os.environ.get(
        "SCHEDVIEW_DAY_OBS",
        (datetime.date.today() - datetime.timedelta(days=1)).isoformat(),
    ).replace("-", "")
)


In [ ]:
"""
# To render this notebook to html, in the current directory with the current python environment

export SCHEDVIEW_VISIT_ORIGIN=lsstcam
export SCHEDVIEW_DAY_OBS='20250720'
jupyter nbconvert \
    --to html \
    --execute \
    --no-input \
    --ExecutePreprocessor.kernel_name=python3 \
    --ExecutePreprocessor.startup_timeout=3600 \
    --ExecutePreprocessor.timeout=3600 \
    svprogress.ipynb

# The public USDF page served at
#    https://s3df.slac.stanford.edu/data/rubin/sim-data/schedview/reports/svprogress/lsstcam/2025/07/20/svprogress_2025-07-20.html
# is found at
#    /sdf/group/rubin/web_data/sim-data/schedview/reports/svprogress/lsstcam/2025/07/20/svprogress_2025-07-20.html

# You can set up the conda environment used to automatically generate these pages with
source /sdf/group/rubin/sw/w_latest/loadLSST.sh
conda activate /sdf/data/rubin/shared/scheduler/envs/prenight
"""
pass

In [ ]:
import os
import sys

#sched_source = 'env'
sched_source = 'shared'
#sched_source = 'devel'
match sched_source:
    case 'shared':
        if os.path.exists('/sdf/data/rubin/shared/scheduler/packages'):
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/uranography-1.2.1")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.11.1.dev1+g2e68149")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_sim-2.2.4")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/schedview-0.18.1.dev19+gad78bef")
    case 'devel':
        if os.path.exists('/sdf/data/rubin/user/neilsen/devel'):
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_scheduler")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_sim")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/schedview")
    case _:
        # Use whatever is in the kernel python environment
        pass

In [ ]:
from schedview.util import config_logging_for_reports
import logging
config_logging_for_reports(logging.ERROR)

In [ ]:
import calendar
import datetime
import os
import sqlite3
import sys
from collections import defaultdict, OrderedDict
from pathlib import Path

import astropy
import astropy.units as u
import bokeh
import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting
import bokeh.transform
import cartopy
import colorcet
import healpy
import healpy as hp
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from erfa import ErfaWarning
from astropy.coordinates import SkyCoord, get_body
from astropy.time import Time
from astropy.visualization import ZScaleInterval
from IPython.display import HTML, display, Markdown, HTML
from lsst.resources import ResourcePath

In [ ]:
usdf_sim_data_dir = "/sdf/data/rubin/shared/rubin_sim_data"
if os.path.exists(usdf_sim_data_dir):
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

In [ ]:
import schedview.collect.consdb
import schedview.collect.nightreport
import schedview.collect.timeline
import rubin_scheduler
import rubin_scheduler.site_models
import rubin_scheduler.utils
import schedview.compute.maf
import schedview.compute.astro
import schedview.compute.visits
import schedview.compute.nightreport
import schedview.plot.survey_skyproj
import schedview.plot.visitmap
import schedview.plot.nightreport
import schedview.collect.visits
import schedview.collect.sv
import rubin_sim.sim_archive
import uranography
from rubin_sim.data import get_baseline
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_sim import maf
from schedview.compute.camera import LsstCameraFootprintPerimeter
from uranography.api import Planisphere, make_zscale_linear_cmap
from schedview import DayObs
from rubin_scheduler.utils.consdb import KNOWN_INSTRUMENTS

In [ ]:
if not Path('/sdf/group/rubin/web_data/sim-data').exists():
    schedview.collect.visits.OPSIMDB_TEMPLATE = str(
        Path.home().joinpath("Data/sim-data/sims_featureScheduler_runs{sim_version}/baseline/baseline_v{sim_version}_10yrs.db")
    )

In [ ]:
# Validate the input
import re

assert visit_origin in KNOWN_INSTRUMENTS or visit_origin == 'baseline' or re.match(r"^\d+\.\d+$", visit_origin) is not None

In [ ]:
# Degraded IERS accuracy is never going to be important for these figures.

# If IERS degraded accuracy encountered, don't fail, just keep going.
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

In [ ]:
bokeh.io.output_notebook(hide_banner=True)

In [ ]:
%matplotlib inline

In [ ]:
ORIGIN_TELESCOPE = defaultdict(
    np.array(['Simonyi']).item,
    {'latiss': 'AuxTel'}
)

In [ ]:
observatory = ModelObservatory(no_sky=True)
timezone = "Chile/Continental"
telescope = ORIGIN_TELESCOPE[visit_origin]
from_opsim = visit_origin not in KNOWN_INSTRUMENTS
science_programs_to_include = ['BLOCK-365']
telescope = "AuxTel" if visit_origin.lower()=="latiss" else "Simonyi"

In [ ]:
day_obs = DayObs.from_date(day_obs)
observatory.mjd = day_obs.mjd + 1 - observatory.location.lon.deg/360 ;# The approximate middle of the night

# Science Verification Survey Progress

### Metric maps

Each subplot represents the map of a metric a different filter, presented in a [Lambert Azimuthal Equal Area Projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection), centered at the south celestial pole.

Annotations are similar to those of the "Visit Map" above:

 - The blue backdrop represents the value of the metric.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
 - The blue line (oval) shows the plane of the Milky Way.
 - Broken red ovals show a zenith distince of 70° (airmass=2.9) at morning and evening 12° twilight.
   - The dashed portions of each red oval show the extent of what is observable at all during the night.
   - As time progress through the night, the ZD circle maintains its shape, moving counter-clockwise around the pole over the course of the night.

In [ ]:
conditions = observatory.return_conditions()

In [ ]:
visits = schedview.collect.sv.read_visits(day_obs)

### Accumulated numbers of visits

In [ ]:
fig = schedview.plot.survey_skyproj.create_metric_map_grid(
    maf.CountMetric(col='fiveSigmaDepth', metric_name="Numbers of visits"),
    visits.loc[np.isfinite(visits['fiveSigmaDepth']), :],
    observatory,
    nside=32,
)

## DDF Cadence

In [ ]:
time_window_duration=90

In [ ]:
try:
    ddf_visits = schedview.collect.visits.read_ddf_visits(day_obs, visit_origin, num_nights=time_window_duration)
    ddf_visits = ddf_visits.loc[ddf_visits['science_program'].isin(science_programs_to_include), :]
except:
    ddf_visits = []

if len(ddf_visits) > 0:
    nightly_ddf = schedview.compute.visits.accum_stats_by_target_band_night(ddf_visits, target_column='field_name')
    cadence_plots = schedview.plot.create_cadence_plot(
        nightly_ddf, day_obs.mjd - time_window_duration, day_obs.mjd, target_column='field_name'
    )
    bokeh.io.show(cadence_plots)
else:
    print("No DDF visits")


The y-axis (height of the vertical bars) represents the accumulated effective exposure time, $t_{\mbox{eff}}$ (as defined [DMTN-296](https://dmtn-296.lsst.io/)) accumulated over all exposures on the field for the night, colored by filter.

## Hourglass plots

In the hourglass plots that follow, each horizontal band represents a timeline for a day of a month. The gray background shows twilight, while the black background is fully night.
The moon rises or sets where the dotted line crosses each night's horizontal line, and transits where the thick solid line crosses it.

The quantities shown here are just a few examples. Similar hourglass plots can be added for any quantity that varies by visit, and whose value can be mapped to a scalar or category.


In [ ]:
class SingularPassMetric(maf.metrics.PassMetric):
    def run(self, data_slice, slice_point=None):
        # PassMetric returns all columns, but we need just the requested one.
        return data_slice[self.col_name_arr[0]]

In [ ]:
year_months = []
first_date, last_date = [DayObs.from_time(m).date for m in visits.observationStartMJD.describe()[['min', 'max']]]
for dt in range((last_date - first_date).days + 1):
    this_date = first_date + datetime.timedelta(dt)
    year_month = (this_date.year, this_date.month)
    if year_month not in year_months:
        year_months.append(year_month)

In [ ]:
def plot_hourglass_metric(visits, metric, name, constraint="", plot_dict={}, plotter=maf.plots.MonthHourglassPlot):
    seeing_bundle = maf.metric_bundles.MetricBundle(
        metric=metric,
        slicer=maf.slicers.VisitIntervalSlicer(),
        constraint=constraint,
        plot_dict=plot_dict,
    )
    schedview.compute.maf.compute_metric(visits, seeing_bundle)
    # If we pass a list of plot_funcs to the MetricBundle, they all
    # get the same plot_dict, an so same title. We want different
    # titles for each month, so iterate here:
    for year, month in year_months:
        seeing_bundle.plot_dict['title'] = f"{name} for {calendar.month_name[month]}, {year}"
        seeing_bundle.plot_funcs = [plotter(month, year)]
        seeing_bundle.plot()


### Seeing

In [ ]:
plot_hourglass_metric(visits, maf.metrics.MedianMetric("seeingFwhmEff"), "seeingFwhmEff")

### Hour Angle

In [ ]:
plot_hourglass_metric(visits, maf.metrics.MedianMetric("HA"), "Hour Angle", plot_dict={"cmap": plt.get_cmap("coolwarm"), "color_limits": (-4.5, 4.5)})

### Band

In [ ]:
plot_dict = {
    "cmap": plt.get_cmap("Set1"),
    "assigned_colors": OrderedDict(
        [("u", 1), ("g", 2), ("r", 4), ("i", 7), ("z", 0), ("y", 3)]
    ),
    "legend_ncols": 1,
    "legend_loc": (1.01, 0.5),
    "legend_bbox_to_anchor": (1.01, 0.0),
    "legend": True,
}

plot_hourglass_metric(visits, SingularPassMetric('band'), "Band", plot_dict=plot_dict, plotter=maf.plots.MonthHourglassCategoricalPlot)